In [1]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import utils.data_processing as dp
import utils.conversion as conv
import copy

from plot_config import *

In [2]:
def plot_fd_pl_one_plot(res_list_or_dict, y_name, ax1_xlabel, ax2_xlabel, label_local_list, N_path, res_keys=None):

    # Define GridSpec layout for the two plots (4 columns: 3 for the first plot, 1 for the second)
    fig = plt.figure(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])  # 3/4 width for the first plot, 1/4 for the second

    # First plot: 3/4 of the width
    ax1 = fig.add_subplot(gs[0, :-1])  # Use the first 3 columns for the first plot

    iterator = dp.iterator_from_list_or_dict(res_list_or_dict, keys=res_keys)
    for i, key in iterator:
        
        tabl = res_list_or_dict[key]['tabl'] if isinstance(res_list_or_dict, dict) else res_list_or_dict[i]['tabl']

        label_local = label_local_list[i]

        if (y_name == 'qx_norm') and 'qx_norm' not in tabl.columns:
            tabl['qx_norm'] = tabl['qx']/(tabl['qx'].sum()/N_path)

        ax1.plot(tabl.index, tabl['qx_norm'], label=label_local,
                color=colors[i],
                linestyle=linestyles[i],
                linewidth=2.)
        
    # Second plot: 1/4 of the width
    ax2 = fig.add_subplot(gs[0, -1])  # Use the last column for the second plot

    iterator = dp.iterator_from_list_or_dict(res_list_or_dict, keys=res_keys)
    test_range = [key for (_, key) in iterator]
    PL_list = [res_list_or_dict[key]['tabl']['Pin'][0] for key in res_list_or_dict.keys()] if isinstance(res_list_or_dict, dict) else [res_list_or_dict[i]['tabl']['Pin'][0] for i in range(len(res_list_or_dict))]

    ax2.plot(np.array(test_range)*1000, PL_list, color=colors[-1])

    ax1.set_xlabel(ax1_xlabel)
    ax1.set_ylabel('Normalized flow rate [-]')
    ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

    # Adding the legend below the first plot
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False, handlelength=2.5)

    ax2.set_xlabel(ax2_xlabel)
    ax2.set_ylabel('PL [Pa]')
    ax2.grid()

    # Adjust the layout to ensure alignment of the plots
    plt.subplots_adjust(wspace=0.1)  # Adjust horizontal space between plots

    # Ensure that both y-axes are aligned
    # ax1.get_shared_y_axes().join(ax1, ax2)

    # Adjust the layout to ensure better spacing between plots
    plt.subplots_adjust(wspace=0.3)  # Increase horizontal space between plots

    return fig, ax1, ax2

In [3]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

option = ''
option = (option + '_') if option != '' else option

res_dict_h = dp.load_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_{option}res_dict_h")
res_dict_w = dp.load_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_{option}res_dict_w")

h_riser_range = res_dict_h['parameters']['x_range']
w_riser_range = res_dict_w['parameters']['x_range']

trash = res_dict_h.pop('parameters', None)
trash = res_dict_w.pop('parameters', None)

In [ ]:
fig, ax1, ax2 = plot_fd_pl_one_plot(dict(reversed(res_dict_h.items())), 'qx_norm', 'Panel number[-]', '$h_{riser}$ [mm]', label_local_list=[f'$h_{{riser}}$ = {round(key*1000,1)} mm' for key in h_riser_range][::-1], N_path=6)

In [ ]:
data[('h','worst')]['min_qx_norm']

In [63]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-V4.5_multiparametric_3.png', dpi=600)

In [ ]:
fig, ax1, ax2 = plot_fd_pl_one_plot(dict(reversed(res_dict_w.items())), 'qx_norm', 'Panel number[-]', '$w_{riser}$ [mm]', label_local_list=[f'$w_{{riser}}$ = {round(key*1000,1)} mm' for key in w_riser_range][::-1], N_path=6)

In [65]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-V4.5_multiparametric_4.png', dpi=600)

## MANIFOLD

In [12]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

option = 'MANIFOLD'
option = (option + '_') if option != '' else option

res_dict_man = dp.load_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_{option}res_dict_man")

D_man_range = res_dict_man['parameters']['x_range']

trash = res_dict_man.pop('parameters')

In [ ]:
fig, ax1, ax2 = plot_fd_pl_one_plot(dict(reversed(res_dict_man.items())), 'qx_norm', 'Panel number[-]', '$D_{man}$ [mm]', label_local_list=[f'$D_{{man}}$ = {round(key*1000,1)} mm' for key in D_man_range][::-1], N_path=6)

In [75]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-V4.5_multiparametric_MAN_2.png', dpi=600)

## MAP

In [5]:
combi = [('h', 'temperate'), ('w', 'temperate'), ('h', 'cold'), ('w', 'cold'), ('man', 'temperate'), ('man','cold')]

option_list = ['','','COLD','COLD','MANIFOLD','MANIFOLD_COLD']
option_list = [option + '_' if option != '' else option for option in option_list]

full_list = [dp.load_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_{option}res_dict_{letter}") for letter, option in zip([combi[i][0] for i in range(len(combi))], option_list)]
full_dict = {combi[i] : full_list[i] for i in range(len(combi))}

In [6]:
def gini_coefficient(Q: np.ndarray) -> float:
    """
    Calculate the Gini coefficient for an array of flow rates.
    
    Parameters:
        Q (np.ndarray): A 1D numpy array containing the flow rates in each path.
        
    Returns:
        float: The Gini coefficient, ranging from 0 (perfectly equal) to 1 (maximum inequality).
    """
    # Ensure Q is a 1D numpy array
    Q = np.sort(Q.flatten())  # Sort the array to ensure ascending order
    N_path = len(Q)
    mean_Q = np.mean(Q)
    
    # If mean is zero (all elements are zero), return 0 (indicating homogeneity)
    if mean_Q == 0:
        return 0.0
    
    # Compute Gini coefficient
    gini_sum = np.sum([np.abs(Q[i] - Q[j]) for i in range(N_path) for j in range(N_path)])
    G = gini_sum / (2 * N_path**2 * mean_Q)
    
    return G

In [7]:
data = {}
N_path = 6

for key, res in full_dict.items():
    x_range = res['parameters']['x_range']
    gini_coeffs = []
    PL_list = []
    min_qx_norm = []
    for key2, res2 in res.items():
        if key2 == 'parameters':
            continue
        gini_coeffs.append(gini_coefficient(res2['tabl']['qx'].values))
        PL_list.append(res2['tabl']['Pin'][0])
        min_qx_norm.append((res2['tabl']['qx']/(res2['tabl']['qx'].sum()/N_path)).min())
    
    data[key] = {'x_range': x_range, 'gini_coeffs': gini_coeffs, 'PL_list': PL_list, 'min_qx_norm' : min_qx_norm}

for key in ['h', 'w', 'man']:
    data[(key, 'worst')] = {
        'x_range': data[(key, 'temperate')]['x_range'],
        'gini_coeffs': data[(key, 'temperate')]['gini_coeffs'],
        'PL_list': data[(key, 'cold')]['PL_list'],
        'min_qx_norm': data[(key, 'temperate')]['min_qx_norm']
    }

In [ ]:

# Sample data points (replace these with your data)
x_data = data[('h', 'worst')]['PL_list']
y_data = data[('h', 'worst')]['gini_coeffs']

# Define the inverse or power-law-like function
def inverse_model(x, a, b, c):
    return a / (x + b) + c

print(x)
print(y)

# Fit the model to the data
params, _ = sco.curve_fit(inverse_model, x_data, y_data, p0=[0.5, 1000, 0.05], maxfev=10000, bounds=(0, np.inf))

# Generate interpolated values
x_interp = np.linspace(min(x_data), max(x_data), 100)  # Fine grid for smooth curve
y_interp = inverse_model(x_interp, *params)

# Plot original data and interpolated curve
plt.plot(x_data, y_data, 'o', label="Original Data")
plt.plot(x_interp, y_interp, '-', label="Interpolated Curve")
plt.legend()
plt.xlabel("PL_list")
plt.ylabel("Gini Coefficients")
plt.show()

In [ ]:
# Define the inverse function for fitting
def inverse_model(x, a, b, c):
    return a / (x + b) + c

# Define the linear function for fitting
def linear_model(x, m, c):
    return m * x + c

# Initialize the plot with given configuration
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

# Plot and interpolate each curve
for i, (key, label) in enumerate(zip(['h', 'w', 'man'], ['Decreasing riser height from 3.9 mm to 0.9 mm', 'Decreasing riser width from 8.9 mm to 1.1 mm', 'Increasing manifold diameter from 18 mm to 39.6 mm'])):
    x = np.array(data[(key, 'worst')]['PL_list'])
    y = np.array(data[(key, 'worst')]['gini_coeffs'])
    
    # Select model based on the curve type
    if key in ['h', 'w']:  # Inverse shape for 'h' and 'w'
        params, _ = sco.curve_fit(inverse_model, x, y, p0=[0.5, 1000, 0.05], maxfev=10000, bounds=(0, np.inf))
        x_interp = np.linspace(min(x), max(x), 100)
        y_curve = inverse_model(x_interp, *params)
    else:  # Linear shape for 'man'
        params, _ = sco.curve_fit(linear_model, x, y)
        x_interp = np.linspace(min(x), max(x), 100)
        y_curve = linear_model(x_interp, *params)
    
    # Plot interpolated curve on the specified axis
    ax1.plot(x_interp, y_curve, label=label,
                                color=colors[i],
                                linestyle=linestyles[i],
                                linewidth=1.5
    )

    ax1.scatter(x, y, color=colors[i], s=50, marker=markers[i])
    
    # Calculate midpoint for arrow placement
    midpoint = len(x_interp) // 2
    if key in ['h', 'w']:
        x_start, y_start = x_interp[midpoint - 1], y_curve[midpoint - 1]
        x_end, y_end = x_interp[midpoint], y_curve[midpoint]
    else:  # Reverse direction for the third row
        x_start, y_start = x_interp[midpoint], y_curve[midpoint]
        x_end, y_end = x_interp[midpoint - 1], y_curve[midpoint - 1]
    
    # Add larger and more visible arrow annotation in the middle of the curve
    # ax1.annotate(
    #     '', xy=(x_end, y_end), xytext=(x_start, y_start),
    #     arrowprops=dict(arrowstyle='-|>', color=colors[i], lw=2, mutation_scale=15)
    # )

    min_qx_norm = np.array(data[(key, 'worst')]['min_qx_norm'])
    # Find the first index where min_qx_norm exceeds 0.9
    bracket_index = np.argmax(min_qx_norm > 0.8)
    bracket_x = x[bracket_index]
    bracket_y = y[bracket_index]

    # Add bracket symbol at the first point where min_qx_norm > 0.9
    # ax1.annotate(
    #     ']', xy=(bracket_x, bracket_y), xytext=(bracket_x, bracket_y + 0.02),
    #     ha='center', va='center', fontsize=14, color=colors[i], fontweight='bold'
    # )

        # Add bracket symbol at the first point where min_qx_norm > 0.9
    if i < 2:  # For the first two curves
        ax1.annotate(
            ']', xy=(bracket_x, bracket_y), xytext=(bracket_x, bracket_y),
            ha='center', va='center', fontsize=18, color=colors[i], fontweight='bold',
            rotation=180
        )
    else:  # For the third curve
        ax1.annotate(
            ']', xy=(bracket_x, bracket_y), xytext=(bracket_x+100, bracket_y + 0.02),
            ha='center', va='center', fontsize=18, color=colors[i], fontweight='bold',
            rotation=90
        )

# Configure the plot
ax1.set_xlabel("Pressure loss at $T_m = -12^\circ C$ [Pa]")
ax1.set_ylabel("Gini Coefficient at $T_m = 25^\circ C$ [-]")
ax1.grid(True)

ax1.set_ylim(0, 0.55)

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=1, frameon=False)

# Show the plot
plt.show()

In [28]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-Gini.png', dpi=600)